# Entrenar un modelo de regresion con AutoML

In [ ]:
# mostrar info de azure-ai-ml

! pip show azure-ai-ml

## Conectar a workspace

In [3]:
# conectar
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(f"Conectado al Workspace: {ml_client.workspace_name}")


Found the config file in: .\config.json
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


Conectado al Workspace: naturgyml4


## Preparar datos

Usar losa datos de diabetes.

Recuperar el activo de datos especificando su nombre `diabetes-training-table` y versión `1`. 

In [4]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

# Crea un conjunto de datos basado en los archivos en la carpeta de datos local
my_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml:housing-data-mltable-ric-2:1")

## Configurar el job AutoML

In [9]:
from azure.ai.ml import automl

# Configurar el job de clasificación
regression_job = automl.regression(
    compute="ricardoenm",
    experiment_name="auto-ml-reg-housing-ric",
    training_data=my_training_data_input,
    target_column_name="price",
    primary_metric="r2_score",
    n_cross_validations=5,
    enable_model_explainability=True,
    featurization={
        "mode": "custom",
        "column_purpose": {
            'date': 'drop',
            'street': 'drop',
            'city': 'drop',
            'statezip': 'drop',
            'country': 'drop'
        }
    },
)

# Establecer los límites
regression_job.set_limits(
    timeout_minutes=60, 
    trial_timeout_minutes=20, 
    max_trials=5,
    enable_early_termination=True,
    
)

# Establecer las propiedades de entrenamiento
regression_job.set_training(
    # blocked_training_algorithms=["Lasso"], 
    enable_onnx_compatible_models=True
)

## Ejecutar el job AutoML

In [8]:
# Enviar el job
returned_job = ml_client.jobs.create_or_update(
    regression_job
)  

# URL
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Monitor your job at https://ml.azure.com/runs/green_worm_cck8f9zwjb?wsid=/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourcegroups/naturgy/workspaces/naturgyml4&tid=8b85184d-f6f0-4ace-80dd-c3bda5a6c156
